In [ ]:
from PIL import Image
from IPython.display import display

In [ ]:
tmp_dir="../tmp"
resu_dir="../Resultat"

In [ ]:
date=20221130
heure=1500

In [ ]:
# IR traitement
# Conversion en 8b
#gdal_translate -ot byte -scale 0 330 0 255 /net/stockagecms/ftp/pub/archipel/dataspace/msg/${date}/*globeMir108GT*${date}_${heure}* ${tmp_dir}/tmp_fic_ir.tif
!gdal_translate -ot byte -scale 0 330 0 255 ../data/*globeMir108GT*{date}_{heure}* {tmp_dir}/tmp_fic_ir.tif

In [ ]:
ll ../data

In [ ]:
# Renforcement contraste image IR
min=20
max=240
gamma=1.4

In [ ]:
!gdal_calc.py --quiet --overwrite -A {tmp_dir}/tmp_fic_ir.tif \
--calc="254*(1/254.*numpy.minimum(254,numpy.maximum(0,254*(numpy.float32(A)-{min})/(numpy.float32({max}-{min})))))**(1/numpy.float32({gamma}))" \
--outfile={tmp_dir}/tmp_fic_ir_gamma.tif

In [ ]:
# Creation bande alpha par traitement de l'image IR
s1 = 120
p1 = 20
s2 = 120
p2 = 20

In [ ]:
import numpy
import matplotlib.pyplot as plt

x = numpy.linspace(0, 255, 100)
y = numpy.floor((1.+numpy.tanh((x-s1)/(p1)))*254./2.)
y2 = numpy.floor((1.+numpy.tanh((x-s2)/(p2)))*254./2.)
y3 = numpy.maximum(y,y2)

plt.plot(x, y)
plt.plot(x, y2)
plt.plot(x, y3)

plt.show()

In [ ]:
!gdal_calc.py --quiet --overwrite -A {tmp_dir}/tmp_fic_ir.tif \
--calc="numpy.maximum(numpy.floor((1.+numpy.tanh((numpy.float32(A)-numpy.float32({s1}))/(numpy.float32({p1}))))*254./2.),numpy.floor((1.+numpy.tanh((numpy.float32(A)-numpy.float32({s2}))/(numpy.float32({p2}))))*254./2.))" \
--outfile={tmp_dir}/tmp_alpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif

In [ ]:
# Creation de l'image IR avec transparence, a partir de l'image IR corrigee en contraste et de la bande alpha calculee
!gdal_merge.py -separate {tmp_dir}/tmp_fic_ir_gamma.tif {tmp_dir}/tmp_fic_ir_gamma.tif {tmp_dir}/tmp_fic_ir_gamma.tif \
{tmp_dir}/tmp_alpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif -o {tmp_dir}/{date}_{heure}_IR_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif

In [ ]:
pwd

In [ ]:
# Creation de l'image composee de l'IR et du globe blue marble
#convert -resize 50% -composite /data/maussa/tmp/blue_marble_msg4.tif ${tmp_dir}/${date}_${heure}_IR_avecalpha_tanh_${s1}_${p1}_et_${s2}_${p2}.tif /
#${resu_dir}/${date}_${heure}_IR_surbluemarble_tanh_${s1}_${p1}_et_${s2}_${p2}.jpg

img = Image.open(tmp_dir + f'/{date}_{heure}_IR_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif', 'r')
fond = Image.open('../meta/fond/blue_marble_msg4.tif', 'r')
fond.paste(img, (0,0), img)
fond.save(resu_dir + f'/{date}_{heure}_IR_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg')

im = Image.open(resu_dir + f'/{date}_{heure}_IR_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg', 'r')
display(im)





In [ ]:
# Conservation des parametres utilises pour faire la bande alpha de l'IR (utilises dans les noms de fichiers)
s1_ir = s1
p1_ir = p1
s2_ir = s2
p2_ir = p2

In [ ]:
# VIS traitement

# Renforcement contraste image VIS
min = 20
max = 245
gamma = 1.2

In [ ]:
!gdal_calc.py --quiet --overwrite -A ./DATA/tif/globeMvis006T*{date}_{heure}* \
--calc="254*(1/254.*numpy.minimum(254,numpy.maximum(0,254*(numpy.float32(A)-{min})/(numpy.float32({max}-{min})))))**(1/numpy.float32({gamma}))" \
--outfile={tmp_dir}/tmp_fic_vis_gamma.tif

In [ ]:
# Creation bande alpha par traitement de l'image VIS
s1 = 55
p1 = 110
s2 = 215
p2 = 120

In [ ]:
import numpy
import matplotlib.pyplot as plt

x = numpy.linspace(0, 255, 100)
y1 = numpy.floor((1.+numpy.tanh((x-s1)/(p1)))*254./2.)
y2 = numpy.floor((1.+numpy.tanh((254-x-s2)/(p2)))*254./2.)
y3 = numpy.maximum(y1,y2)

fig, ax = plt.subplots()
ax.plot(x, y1, label="seuillage 1")
ax.plot(x, y2, label="seuillage 2")
ax.plot(x, y3, label="seuillage du max")
ax.legend()

plt.show()

In [ ]:
!gdal_calc.py --quiet --NoDataValue 0 --overwrite -A ./DATA/tif/globeMvis006T*{date}_{heure}* \
--calc="numpy.maximum(numpy.floor((1.+numpy.tanh((numpy.float32(254-A)-numpy.float32({s2}))/(numpy.float32({p2}))))*254./2.),numpy.floor((1.+numpy.tanh((numpy.float32(A)-numpy.float32({s1}))/(numpy.float32({p1}))))*254./2.))" \
--outfile={tmp_dir}/tmp_alpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif

In [ ]:
# Creation de l'image VIS avec transparence, a partir de l'image VIS corrigee en contraste et de la bande alpha calculee
!gdal_merge.py -separate {tmp_dir}/tmp_fic_vis_gamma.tif {tmp_dir}/tmp_fic_vis_gamma.tif {tmp_dir}/tmp_fic_vis_gamma.tif \
{tmp_dir}/tmp_alpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif -o {tmp_dir}/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif

In [ ]:
# Creation de l'image composee de la VIS et du globe blue marble
#convert -composite /data/maussa/tmp/blue_marble_msg4.tif ${tmp_dir}/${date}_${heure}_VIS_avecalpha_tanh_${s1}_${p1}_et_${s2}_${p2}.tif /
#${resu_dir}/${date}_${heure}_VIS_surbluemarble_tanh_${s1}_${p1}_et_${s2}_${p2}.jpg

img = Image.open(tmp_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif', 'r')
fond = Image.open('./data/meta/blue_marble_msg4.tif', 'r')
fond.paste(img, (0,0), img)
fond.save(resu_dir + f'/{date}_{heure}_VIS_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg')

im = Image.open(resu_dir + f'/{date}_{heure}_VIS_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg', 'r')
display(im)


In [ ]:
img = Image.open(tmp_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif', 'r')
fond = Image.open(resu_dir + f'/{date}_{heure}_IR_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg')
fond.paste(img, (0,0), img)
fond.save(resu_dir + f'/{date}_{heure}_IR_VIS_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg')
im = Image.open(resu_dir + f'/{date}_{heure}_IR_VIS_surbluemarble_tanh_{s1}_{p1}_et_{s2}_{p2}.jpg', 'r')
display(im)

In [ ]:
# Compilation IR et VIS
#convert -composite ${tmp_dir}/${date}_${heure}_IR_avecalpha_tanh_${s1_ir}_${p1_ir}_et_${s2_ir}_${p2_ir}.tif /
#${tmp_dir}/${date}_${heure}_VIS_avecalpha_tanh_${s1}_${p1}_et_${s2}_${p2}.tif /
#${tmp_dir}/${date}_${heure}_VIS_avecalpha_tanh_${s1}_${p1}_${s2}_${p2}_et_IR_avecalpha_tanh_${s1_ir}_${p1_ir}_${s2_ir}_${p2_ir}.tif

# Etape 1 :
img = Image.open(tmp_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}.tif', 'r')
fond = Image.open(tmp_dir + f'/{date}_{heure}_IR_avecalpha_tanh_{s1_ir}_{p1_ir}_et_{s2_ir}_{p2_ir}.tif', 'r')
fond.paste(img, (0,0), img)
fond.save(resu_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}_et_{date}_{heure}_IR_avecalpha_tanh_{s1_ir}_{p1_ir}_et_{s2_ir}_{p2_ir}.tif')

# Etape 2 :
img2 = Image.open(resu_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}_et_{date}_{heure}_IR_avecalpha_tanh_{s1_ir}_{p1_ir}_et_{s2_ir}_{p2_ir}.tif', 'r')
fond2 = Image.open('./data/meta/blue_marble_msg4.tif', 'r')
fond2.paste(img2, (0,0), img2)
fond2.save(resu_dir + f'/{date}_{heure}_VIS_tanh_{s1}_{p1}_{s2}_{p2}_et_IR_tanh_{s1_ir}_{p1_ir}_{s2_ir}_{p2_ir}_sur_bluemarble.jpg')
#im_final = Image.open(resu_dir + f'/{date}_{heure}_VIS_tanh_{s1}_{p1}_{s2}_{p2}_et_IR_tanh_{s1_ir}_{p1_ir}_{s2_ir}_{p2_ir}_sur_bluemarble.jpg', 'r')
#display(im_final)

#convert -gamma 1.1 -composite /data/maussa/tmp/blue_marble_msg4.tif /
#${tmp_dir}/${date}_${heure}_VIS_avecalpha_tanh_${s1}_${p1}_${s2}_${p2}_et_IR_avecalpha_tanh_${s1_ir}_${p1_ir}_${s2_ir}_${p2_ir}.tif /
#${resu_dir}/${date}_${heure}_VIS_tanh_${s1}_${p1}_${s2}_${p2}_et_IR_tanh_${s1_ir}_${p1_ir}_${s2_ir}_${p2_ir}_sur_bluemarble.jpg

In [ ]:
img2 = Image.open(resu_dir + f'/{date}_{heure}_VIS_avecalpha_tanh_{s1}_{p1}_et_{s2}_{p2}_et_{date}_{heure}_IR_avecalpha_tanh_{s1_ir}_{p1_ir}_et_{s2_ir}_{p2_ir}.tif', 'r')
fond2 = Image.open('./data/meta/blue_marble_msg4.tif', 'r')
fond2.paste(img2, (0,0), img2)
fond2.save(resu_dir + f'/{date}_{heure}_VIS_tanh_{s1}_{p1}_{s2}_{p2}_et_IR_tanh_{s1_ir}_{p1_ir}_{s2_ir}_{p2_ir}_sur_bluemarble.jpg')

In [ ]:
display(img)
display(img2)